# Sistema de ML Multi-Módulo
Seleccione el módulo a ejecutar:
1. Predicción de Series Temporales (Ventas)
2. Clasificación de Imágenes (MNIST)
3. Análisis de Sentimientos
4. Generación de Contenido

In [ ]:
!pip install pandas numpy matplotlib seaborn scikit-learn tensorflow


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv2D, MaxPooling2D, Flatten, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import mnist
import io

# Configurar semilla para reproducibilidad
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)


## Función para elegir qué módulo ejecutar

In [3]:
def ejecutar_modulo(opcion):
    if opcion == 1:
        print("\n=== EJECUTANDO MÓDULO 1: PREDICCIÓN DE SERIES TEMPORALES ===")
        modulo_prediccion_ventas()
    elif opcion == 2:
        print("\n=== EJECUTANDO MÓDULO 2: CLASIFICACIÓN DE IMÁGENES ===")
        modulo_clasificacion_imagenes()
    elif opcion == 3:
        print("\n=== EJECUTANDO MÓDULO 3: ANÁLISIS DE SENTIMIENTOS ===")
        modulo_analisis_sentimientos()
    elif opcion == 4:
        print("\n=== EJECUTANDO MÓDULO 4: GENERACIÓN DE CONTENIDO ===")
        modulo_generacion_contenido()
    else:
        print("Opción no válida")


## Módulo 1: Predicción de Series Temporales

In [6]:
def modulo_prediccion_ventas():
    print("=== GENERANDO DATASET DE VENTAS ===")
    # Generar fechas para un año (2023)
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)
    dates = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]
    
    # Crear diccionario para días festivos en España (simplificado)
    holidays = {
        '2023-01-01': 'Año Nuevo',
        '2023-01-06': 'Reyes Magos',
        '2023-04-07': 'Viernes Santo',
        '2023-05-01': 'Día del Trabajo',
        '2023-08-15': 'Asunción',
        '2023-10-12': 'Día de la Hispanidad',
        '2023-11-01': 'Todos los Santos',
        '2023-12-06': 'Día de la Constitución',
        '2023-12-08': 'Inmaculada Concepción',
        '2023-12-25': 'Navidad'
    }
    
    # Función para determinar la temporada
    def get_season(date):
        month = date.month
        if month in [12, 1, 2]:
            return 'Invierno'
        elif month in [3, 4, 5]:
            return 'Primavera'
        elif month in [6, 7, 8]:
            return 'Verano'
        else:
            return 'Otoño'
    
    # Generar datos para cada día
    data = []
    for date in dates:
        day_of_week = date.weekday()  # 0-6 (lunes-domingo)
        date_str = date.strftime('%Y-%m-%d')
        
        # Determinar si es festivo
        is_holiday = 1 if date_str in holidays else 0
        
        # Temporada
        season = get_season(date)
        
        # Temperatura (simulada según temporada)
        if season == 'Invierno':
            temp = np.random.normal(10, 5)
        elif season == 'Primavera':
            temp = np.random.normal(18, 5)
        elif season == 'Verano':
            temp = np.random.normal(28, 5)
        else:  # Otoño
            temp = np.random.normal(15, 5)
        
        # Lluvia (probabilidad varía según temporada)
        if season == 'Invierno' or season == 'Otoño':
            rain = np.random.choice([0, 1], p=[0.6, 0.4])
        else:
            rain = np.random.choice([0, 1], p=[0.8, 0.2])
        
        # Promociones (aleatorias con mayor probabilidad en ciertos meses)
        if date.month in [1, 7]:  # Rebajas
            promo = np.random.choice([0, 1], p=[0.3, 0.7])
        else:
            promo = np.random.choice([0, 1], p=[0.8, 0.2])
        
        # Base de ventas según día de la semana
        if day_of_week < 5:  # Lunes a viernes
            base_sales = np.random.normal(1000, 200)
        else:  # Fin de semana
            base_sales = np.random.normal(1500, 300)
        
        # Modificadores
        if is_holiday:
            base_sales *= np.random.uniform(0.8, 1.2)  # Efecto variable de festivos
        
        if promo:
            base_sales *= np.random.uniform(1.2, 1.6)  # Efecto de promociones
        
        # Efecto de la temperatura (las temperaturas moderadas favorecen las ventas)
        temp_effect = -0.001 * (temp - 22) ** 2 + 1
        base_sales *= temp_effect
        
        # Efecto de la lluvia (reduce ventas)
        if rain:
            base_sales *= np.random.uniform(0.8, 0.95)
        
        # Tendencia anual (ligero crecimiento)
        day_of_year = date.timetuple().tm_yday
        trend = 1 + (day_of_year / 365) * 0.05
        base_sales *= trend
        
        # Picos estacionales
        if date.month == 12:  # Navidad
            base_sales *= np.random.uniform(1.3, 1.8)
        elif date.month == 11 and date.day > 25:  # Black Friday
            base_sales *= np.random.uniform(1.4, 2.0)
        
        # Añadir ruido final
        sales = max(0, int(base_sales * np.random.normal(1, 0.05)))
        
        data.append({
            'fecha': date_str,
            'dia_semana': day_of_week,
            'festivo': is_holiday,
            'temporada': season,
            'temperatura': round(temp, 1),
            'lluvia': rain,
            'promocion': promo,
            'ventas': sales
        })

    # Crear DataFrame
    df = pd.DataFrame(data)

    # One-hot encoding para temporada
    season_dummies = pd.get_dummies(df['temporada'], prefix='temporada')
    df = pd.concat([df, season_dummies], axis=1)

    # Convertir día de la semana a categoría cíclica usando seno y coseno
    df['dia_semana_sin'] = np.sin(2 * np.pi * df['dia_semana'] / 7)
    df['dia_semana_cos'] = np.cos(2 * np.pi * df['dia_semana'] / 7)

    # Guardar el DataFrame a un archivo CSV
    df.to_csv('ventas_tienda_diarias.csv', index=False)

    print(f"\nDataset creado y guardado como 'ventas_tienda_diarias.csv'")
    print(f"Tamaño del dataset: {len(df)} registros")

    # Mostrar algunas estadísticas básicas
    print("\nEstadísticas de ventas:")
    print(df['ventas'].describe())

    print("\n=== ENTRENANDO MODELO DE RED NEURONAL ===")

    # Preparar los datos para la red neuronal
    # Seleccionar características y objetivo
    X = df[['dia_semana_sin', 'dia_semana_cos', 'festivo', 'temperatura',
            'lluvia', 'promocion', 'temporada_Invierno', 'temporada_Otoño',
            'temporada_Primavera', 'temporada_Verano']]
    y = df['ventas']

    # Normalizar los datos
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

    # Dividir en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

    # Construir el modelo de red neuronal
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)  # Capa de salida para la regresión
    ])

    # Compilar el modelo
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    # Configurar early stopping para evitar sobreajuste
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True
    )

    # Entrenar el modelo
    history = model.fit(
        X_train, y_train,
        epochs=100,  # Reducido para que sea más rápido
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )

    # Evaluar el modelo
    train_loss, train_mae = model.evaluate(X_train, y_train, verbose=0)
    test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)

    print(f'\nPérdida en entrenamiento: {train_loss:.4f}')
    print(f'MAE en entrenamiento: {train_mae:.4f}')
    print(f'Pérdida en prueba: {test_loss:.4f}')
    print(f'MAE en prueba: {test_mae:.4f}')

    # Hacer predicciones
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_test_actual = scaler_y.inverse_transform(y_test)

    # Calcular métricas de error en valores originales (no escalados)
    mae = mean_absolute_error(y_test_actual, y_pred)
    rmse = math.sqrt(mean_squared_error(y_test_actual, y_pred))
    r2 = r2_score(y_test_actual, y_pred)

    print(f'\nMAE (valores originales): {mae:.2f}')
    print(f'RMSE (valores originales): {rmse:.2f}')
    print(f'R²: {r2:.4f}')

    # Guardar el modelo entrenado
    model.save('modelo_ventas_tienda.h5')
    print("\nModelo guardado como 'modelo_ventas_tienda.h5'")

    # Función para predecir ventas con nuevos datos
    def predict_sales(dia_semana, festivo, temperatura, lluvia, promocion, temporada):
        """Predecir ventas con nuevos datos"""
        # Convertir día de la semana a valores cíclicos
        dia_semana_sin = np.sin(2 * np.pi * dia_semana / 7)
        dia_semana_cos = np.cos(2 * np.pi * dia_semana / 7)
       
        # Preparar temporada (one-hot encoding)
        temp_invierno = 1 if temporada == 'Invierno' else 0
        temp_otono = 1 if temporada == 'Otoño' else 0
        temp_primavera = 1 if temporada == 'Primavera' else 0
        temp_verano = 1 if temporada == 'Verano' else 0
       
        # Crear array de entrada
        X_new = np.array([[dia_semana_sin, dia_semana_cos, festivo, temperatura,
                          lluvia, promocion, temp_invierno, temp_otono,
                          temp_primavera, temp_verano]])
        
        # Normalizar
        X_new_scaled = scaler_X.transform(X_new)
        
        # Predecir
        y_new_scaled = model.predict(X_new_scaled)
        
        # Desnormalizar
        y_new = scaler_y.inverse_transform(y_new_scaled)
        
        return y_new[0][0]
    
    print("\n=== REALIZANDO ANÁLISIS DE SENSIBILIDAD ===")

    # Análisis 1: Impacto del día de la semana
    dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    ventas_por_dia = []

    for i in range(7):
        ventas = predict_sales(
            dia_semana=i,
            festivo=0,
            temperatura=20.0,
            lluvia=0,
            promocion=0,
            temporada='Primavera'
        )
        ventas_por_dia.append(ventas)

    plt.figure(figsize=(10, 6))
    plt.bar(dias, ventas_por_dia)
    plt.title('Ventas previstas por día de la semana')
    plt.xlabel('Día de la semana')
    plt.ylabel('Ventas previstas')
    plt.savefig('ventas_por_dia.png')
    plt.close()  # Cerrar figura para liberar memoria

    print("- Análisis de impacto por día de la semana completado")

    # Análisis 2: Impacto de la temperatura
    temperaturas = np.arange(0, 41, 2)
    ventas_por_temp = []

    for temp in temperaturas:
        ventas = predict_sales(
            dia_semana=5,  # Sábado
            festivo=0,
            temperatura=temp,
            lluvia=0,
            promocion=0,
            temporada='Verano'
        )
        ventas_por_temp.append(ventas)

    plt.figure(figsize=(10, 6))
    plt.plot(temperaturas, ventas_por_temp, marker='o')
    plt.title('Impacto de la temperatura en las ventas (sábado de verano)')
    plt.xlabel('Temperatura (°C)')
    plt.ylabel('Ventas previstas')
    plt.grid(True)
    plt.savefig('ventas_por_temperatura.png')
    plt.close()  # Cerrar figura para liberar memoria

    print("- Análisis de impacto de temperatura completado")

    # Análisis 3: Comparación entre temporadas
    temporadas = ['Invierno', 'Primavera', 'Verano', 'Otoño']
    ventas_por_temporada = []

    for temporada in temporadas:
        ventas = predict_sales(
            dia_semana=5,  # Sábado
            festivo=0,
            temperatura=20.0,  # Temperatura moderada en todas las temporadas
            lluvia=0,
            promocion=0,
            temporada=temporada
        )
        ventas_por_temporada.append(ventas)

    plt.figure(figsize=(10, 6))
    plt.bar(temporadas, ventas_por_temporada)
    plt.title('Ventas previstas por temporada (sábado sin promoción)')
    plt.xlabel('Temporada')
    plt.ylabel('Ventas previstas')
    plt.savefig('ventas_por_temporada.png')
    plt.close()  # Cerrar figura para liberar memoria

    print("- Análisis de impacto por temporada completado")

    # Análisis 4: Impacto de promociones por temporada
    ventas_con_promo = []
    ventas_sin_promo = []

    for temporada in temporadas:
        # Con promoción
        ventas_cp = predict_sales(
            dia_semana=5,  # Sábado
            festivo=0,
            temperatura=20.0,
            lluvia=0,
            promocion=1,
            temporada=temporada
        )
        ventas_con_promo.append(ventas_cp)
       
        # Sin promoción
        ventas_sp = predict_sales(
            dia_semana=5,  # Sábado
            festivo=0,
            temperatura=20.0,
            lluvia=0,
            promocion=0,
            temporada=temporada
        )
        ventas_sin_promo.append(ventas_sp)

    width = 0.35
    x = np.arange(len(temporadas))

    plt.figure(figsize=(12, 6))
    plt.bar(x - width/2, ventas_sin_promo, width, label='Sin promoción')
    plt.bar(x + width/2, ventas_con_promo, width, label='Con promoción')
    plt.title('Impacto de promociones por temporada (sábado)')
    plt.xlabel('Temporada')
    plt.ylabel('Ventas previstas')
    plt.xticks(x, temporadas)
    plt.legend()
    plt.savefig('impacto_promociones.png')
    plt.close()  # Cerrar figura para liberar memoria

    print("- Análisis de impacto de promociones completado")

    # Análisis 5: Impacto de festivos vs no festivos
    dias_semana = [0, 5]  # Lunes y Sábado
    dias_nombres = ['Lunes', 'Sábado']
    ventas_festivo = []
    ventas_normal = []

    for dia in dias_semana:
        # Día festivo
        ventas_f = predict_sales(
            dia_semana=dia,
            festivo=1,
            temperatura=20.0,
            lluvia=0,
            promocion=0,
            temporada='Primavera'
        )
        ventas_festivo.append(ventas_f)
       
        # Día normal
        ventas_n = predict_sales(
            dia_semana=dia,
            festivo=0,
            temperatura=20.0,
            lluvia=0,
            promocion=0,
            temporada='Primavera'
        )
        ventas_normal.append(ventas_n)

    width = 0.35
    x = np.arange(len(dias_nombres))

    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, ventas_normal, width, label='Día normal')
    plt.bar(x + width/2, ventas_festivo, width, label='Día festivo')
    plt.title('Comparación de ventas en días normales vs festivos')
    plt.xlabel('Día')
    plt.ylabel('Ventas previstas')
    plt.xticks(x, dias_nombres)
    plt.legend()
    plt.savefig('impacto_festivos.png')
    plt.close()  # Cerrar figura para liberar memoria

    print("- Análisis de impacto de días festivos completado")

    # Análisis 6: Impacto de la lluvia en diferentes temporadas
    ventas_con_lluvia = []
    ventas_sin_lluvia = []

    for temporada in temporadas:
        # Con lluvia
        ventas_cl = predict_sales(
            dia_semana=5,  # Sábado
            festivo=0,
            temperatura=20.0,
            lluvia=1,
            promocion=0,
            temporada=temporada
        )
        ventas_con_lluvia.append(ventas_cl)
       
        # Sin lluvia
        ventas_sl = predict_sales(
            dia_semana=5,  # Sábado
            festivo=0,
            temperatura=20.0,
            lluvia=0,
            promocion=0,
            temporada=temporada
        )
        ventas_sin_lluvia.append(ventas_sl)

    width = 0.35
    x = np.arange(len(temporadas))

    plt.figure(figsize=(12, 6))
    plt.bar(x - width/2, ventas_sin_lluvia, width, label='Sin lluvia')
    plt.bar(x + width/2, ventas_con_lluvia, width, label='Con lluvia')
    plt.title('Impacto de la lluvia por temporada (sábado)')
    plt.xlabel('Temporada')
    plt.ylabel('Ventas previstas')
    plt.xticks(x, temporadas)
    plt.legend()
    plt.savefig('impacto_lluvia.png')
    plt.close()  # Cerrar figura para liberar memoria

    print("- Análisis de impacto de lluvia completado")

    # Crear un resumen de los hallazgos
    print("\n=== RESUMEN DE ANÁLISIS DE SENSIBILIDAD ===")
    print(f"1. Día con mayores ventas: {dias[np.argmax(ventas_por_dia)]} ({ventas_por_dia[np.argmax(ventas_por_dia)]:.2f})")
    print(f"2. Temperatura óptima para ventas: {temperaturas[np.argmax(ventas_por_temp)]:.1f}°C ({ventas_por_temp[np.argmax(ventas_por_temp)]:.2f})")
    print(f"3. Temporada con mayores ventas: {temporadas[np.argmax(ventas_por_temporada)]} ({ventas_por_temporada[np.argmax(ventas_por_temporada)]:.2f})")

    # Calcular el aumento promedio debido a promociones
    aumento_promo = [(con - sin) / sin * 100 for con, sin in zip(ventas_con_promo, ventas_sin_promo)]
    print(f"4. Aumento promedio de ventas por promociones: {np.mean(aumento_promo):.2f}%")

    # Calcular el cambio promedio debido a festivos
    cambio_festivo = [(festivo - normal) / normal * 100 for festivo, normal in zip(ventas_festivo, ventas_normal)]
    print(f"5. Cambio promedio en ventas en días festivos: {np.mean(cambio_festivo):.2f}%")

    # Calcular el cambio promedio debido a lluvia
    cambio_lluvia = [(lluvia - no_lluvia) / no_lluvia * 100 for lluvia, no_lluvia in zip(ventas_con_lluvia, ventas_sin_lluvia)]
    print(f"6. Cambio promedio en ventas por lluvia: {np.mean(cambio_lluvia):.2f}%")

    # Exportar resultados a un archivo CSV
    resultados = pd.DataFrame({
        'Variable': ['Día de la semana', 'Temperatura', 'Temporada', 'Promoción', 'Festivo', 'Lluvia'],
        'Mejor valor': [
            dias[np.argmax(ventas_por_dia)],
            f"{temperaturas[np.argmax(ventas_por_temp)]:.1f}°C",
            temporadas[np.argmax(ventas_por_temporada)],
            'Con promoción',
            'Depende del día',
            'Sin lluvia'
        ],
        'Impacto': [
            f"{(max(ventas_por_dia) - min(ventas_por_dia)) / min(ventas_por_dia) * 100:.2f}%%",
            f"{(max(ventas_por_temp) - min(ventas_por_temp)) / min(ventas_por_temp) * 100:.2f}%%",
            f"{(max(ventas_por_temporada) - min(ventas_por_temporada)) / min(ventas_por_temporada) * 100:.2f}%%",
            f"{np.mean(aumento_promo):.2f}%%",
            f"{np.mean(cambio_festivo):.2f}%%",
            f"{np.mean(cambio_lluvia):.2f}%%"
        ]
    })

    resultados.to_csv('resultados_analisis_sensibilidad.csv', index=False)
    print("\nResultados guardados en 'resultados_analisis_sensibilidad.csv'")

    print("\n=== PROCESO COMPLETO FINALIZADO CON ÉXITO ===")
    print("Archivos generados:")
    print("- ventas_tienda_diarias.csv (Dataset)")
    print("- modelo_ventas_tienda.h5 (Modelo entrenado)")
    print("- ventas_por_dia.png (Gráfica)")  
    print("- ventas_por_temperatura.png (Gráfica)")
    print("- ventas_por_temporada.png (Gráfica)")
    print("- impacto_promociones.png (Gráfica)")
    print("- impacto_festivos.png (Gráfica)")
    print("- impacto_lluvia.png (Gráfica)")
    print("- resultados_analisis_sensibilidad.csv (Resumen de resultados)")


## Módulo 2: Clasificación de Imágenes

In [7]:
def modulo_clasificacion_imagenes():
    print("=== CARGANDO DATASET MNIST ===")
    
    # Cargar el dataset MNIST
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    
    # Normalizar los datos
    X_train = X_train.astype('float32') / 255
    X_test = X_test.astype('float32') / 255
    
    # Reshape para CNN
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
    X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
    
    print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]} imágenes")
    print(f"Tamaño del conjunto de prueba: {X_test.shape[0]} imágenes")
    print(f"Forma de cada imagen: {X_train.shape[1:]} píxeles")
    
    # Visualizar algunas imágenes de muestra
    plt.figure(figsize=(10, 5))
    for i in range(10):
        plt.subplot(2, 5, i+1)
        plt.imshow(X_train[i].reshape(28, 28), cmap='gray')
        plt.title(f'Dígito: {y_train[i]}')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('mnist_muestras.png')
    plt.close()
    
    print("- Muestra de imágenes guardada como 'mnist_muestras.png'")
    
    print("\n=== CONSTRUYENDO Y ENTRENANDO CNN PARA MNIST ===")
    
    # Construir modelo CNN para MNIST
    model_cnn = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(10, activation='softmax')  # 10 clases (dígitos 0-9)
    ])
    
    # Compilar el modelo
    model_cnn.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Entrenar el modelo (con menos épocas para ser más rápido)
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    
    history_cnn = model_cnn.fit(
        X_train, y_train,
        batch_size=128,
        epochs=5,  # Pocas épocas para ser rápido
        validation_split=0.1,
        callbacks=[early_stopping],
        verbose=1
    )
    
    # Evaluar el modelo
    test_loss, test_acc = model_cnn.evaluate(X_test, y_test, verbose=0)
    print(f"Precisión en el conjunto de prueba: {test_acc*100:.2f}%")
    
    # Guardar el modelo
    model_cnn.save('modelo_mnist.h5')
    print("Modelo guardado como 'modelo_mnist.h5'")
    
    # Visualizar algunas predicciones
    predictions = model_cnn.predict(X_test[:10])
    predicted_classes = np.argmax(predictions, axis=1)
    
    plt.figure(figsize=(15, 5))
    for i in range(10):
        plt.subplot(2, 5, i+1)
        plt.imshow(X_test[i].reshape(28, 28), cmap='gray')
        color = 'green' if predicted_classes[i] == y_test[i] else 'red'
        plt.title(f'Pred: {predicted_classes[i]}, Real: {y_test[i]}', color=color)
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('mnist_predicciones.png')
    plt.close()
    
    print("- Predicciones guardadas como 'mnist_predicciones.png'")
    
    # Visualizar matriz de confusión
    y_pred = model_cnn.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    confusion_mtx = tf.math.confusion_matrix(y_test, y_pred_classes)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicción')
    plt.ylabel('Real')
    plt.title('Matriz de Confusión')
    plt.savefig('mnist_confusion_matrix.png')
    plt.close()
    
    print("- Matriz de confusión guardada como 'mnist_confusion_matrix.png'")
    
    print("\n=== CLASIFICADOR DE IMÁGENES MNIST FINALIZADO CON ÉXITO ===")
    print("Archivos generados:")
    print("- mnist_muestras.png (Muestra de imágenes)")
    print("- modelo_mnist.h5 (Modelo CNN entrenado)")
    print("- mnist_predicciones.png (Visualización de predicciones)")
    print("- mnist_confusion_matrix.png (Matriz de confusión)")


## Módulo 3: Análisis de Sentimientos

In [8]:
def modulo_analisis_sentimientos():
    print("=== GENERANDO DATASET DE ANÁLISIS DE SENTIMIENTOS ===")
    
    # Generar un pequeño dataset de reseñas de ejemplo
    reseñas = [
        "Este producto es excelente, estoy muy contento con mi compra.",
        "No me gustó nada, es de pésima calidad y no funciona bien.",
        "Buena relación calidad-precio, recomendable.",
        "Decepcionante, no cumple con lo que promete en el anuncio.",
        "Increíble producto, supera todas mis expectativas.",
        "Terrible servicio al cliente, nunca más compraré aquí.",
        "La entrega fue rápida y el producto llegó en perfectas condiciones.",
        "Pésima experiencia de compra, el producto vino dañado.",
        "Me encanta, funciona muy bien y es fácil de usar.",
        "No lo recomiendo, se rompió a los pocos días de usarlo.",
        "Buena calidad de construcción, resistente y duradero.",
        "Demasiado caro para lo que ofrece, hay mejores opciones.",
        "Excelente atención al cliente, resolvieron mi problema rápidamente.",
        "La aplicación es intuitiva y funciona sin problemas.",
        "Inestable y lleno de errores, no puedo utilizarlo correctamente.",
        "Gran producto, vale cada centavo que pagué por él.",
        "No puedo estar más feliz con esta compra, totalmente recomendable.",
        "Decepción total, nada que ver con lo que esperaba.",
        "Perfecto para mis necesidades, justo lo que buscaba.",
        "Mala experiencia, no volveré a confiar en esta marca."
    ]
    
    # Asignar etiquetas manualmente (0: negativo, 1: positivo)
    sentimientos = [
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 1, 0, 1, 1, 0, 1, 0
    ]
    
    # Crear DataFrame
    df_sentimientos = pd.DataFrame({
        'reseña': reseñas,
        'sentimiento': sentimientos
    })
    
    # Guardar a CSV
    df_sentimientos.to_csv('dataset_sentimientos.csv', index=False)
    
    print(f"Dataset creado con {len(df_sentimientos)} reseñas")
    print("- Guardado como 'dataset_sentimientos.csv'")
    
    # Mostrar distribución de sentimientos
    count_sentimientos = df_sentimientos['sentimiento'].value_counts()
    plt.figure(figsize=(8, 6))
    plt.bar(['Negativo', 'Positivo'], [count_sentimientos[0], count_sentimientos[1]])
    plt.title('Distribución de Sentimientos en el Dataset')
    plt.savefig('distribucion_sentimientos.png')
    plt.close()
    
    print("\n=== PREPROCESANDO TEXTO PARA ANÁLISIS DE SENTIMIENTOS ===")
    
    # Tokenización y padding
    max_palabras = 1000
    max_longitud = 100
    
    tokenizer = Tokenizer(num_words=max_palabras, oov_token="<OOV>")
    tokenizer.fit_on_texts(df_sentimientos['reseña'])
    
    secuencias = tokenizer.texts_to_sequences(df_sentimientos['reseña'])
    secuencias_padded = pad_sequences(secuencias, maxlen=max_longitud, padding='post')
    
    # Dividir en conjuntos de entrenamiento y prueba
    X = secuencias_padded
    y = np.array(df_sentimientos['sentimiento'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    print(f"Tamaño del vocabulario: {len(tokenizer.word_index) + 1}")
    print(f"Longitud máxima de secuencia: {max_longitud}")
    
    print("\n=== CONSTRUYENDO MODELO DE ANÁLISIS DE SENTIMIENTOS ===")
    
    # Construir modelo
    model_sentimiento = Sequential([
        Embedding(max_palabras, 16, input_length=max_longitud),
        Dropout(0.2),
        Flatten(),
        Dense(24, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    
    # Compilar modelo
    model_sentimiento.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Entrenar modelo
    history_sentimiento = model_sentimiento.fit(
        X_train, y_train,
        epochs=10,
        validation_split=0.2,
        verbose=1
    )
    
    # Evaluar en conjunto de prueba
    test_loss, test_acc = model_sentimiento.evaluate(X_test, y_test)
    print(f"\nPrecisión en conjunto de prueba: {test_acc*100:.2f}%")
    
    # Guardar modelo
    model_sentimiento.save('modelo_sentimientos.h5')
    print("Modelo guardado como 'modelo_sentimientos.h5'")
    
    # Visualizar curva de aprendizaje
    plt.figure(figsize=(10, 6))
    plt.plot(history_sentimiento.history['accuracy'], label='train')
    plt.plot(history_sentimiento.history['val_accuracy'], label='validation')
    plt.title('Curva de Aprendizaje - Precisión')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión')
    plt.legend()
    plt.savefig('curva_aprendizaje_sentimientos.png')
    plt.close()
    
    # Hacer predicciones con nuevas reseñas
    nuevas_reseñas = [
        "Este producto es una maravilla, lo recomiendo totalmente",
        "Terrible experiencia, no volvería a comprar este producto"
    ]
    
    nuevas_secuencias = tokenizer.texts_to_sequences(nuevas_reseñas)
    nuevas_padded = pad_sequences(nuevas_secuencias, maxlen=max_longitud, padding='post')
    
    predicciones = model_sentimiento.predict(nuevas_padded)
    
    print("\n=== PREDICCIONES CON NUEVAS RESEÑAS ===")
    for i, reseña in enumerate(nuevas_reseñas):
        sentimiento = "Positivo" if predicciones[i][0] > 0.5 else "Negativo"
        confianza = predicciones[i][0] if predicciones[i][0] > 0.5 else 1 - predicciones[i][0]
        print(f"Reseña: '{reseña}'")
        print(f"Predicción: {sentimiento} (confianza: {confianza*100:.2f}%)\n")
    
    print("\n=== ANÁLISIS DE SENTIMIENTOS FINALIZADO CON ÉXITO ===")
    print("Archivos generados:")
    print("- dataset_sentimientos.csv (Dataset)")
    print("- distribucion_sentimientos.png (Gráfica)")
    print("- modelo_sentimientos.h5 (Modelo entrenado)")
    print("- curva_aprendizaje_sentimientos.png (Gráfica)")


## Módulo 4: Generación de Contenido

In [9]:
def modulo_generacion_contenido():
    print("=== IMPLEMENTANDO GENERADOR DE TEXTO SIMPLE ===")
    
    # Texto de ejemplo (fragmento de Don Quijote)
    texto_ejemplo = """
    En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor. Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lantejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda. El resto della concluían sayo de velarte, calzas de velludo para las fiestas, con sus pantuflos de lo mesmo, y los días de entresemana se honraba con su vellorí de lo más fino. Tenía en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la podadera. Frisaba la edad de nuestro hidalgo con los cincuenta años; era de complexión recia, seco de carnes, enjuto de rostro, gran madrugador y amigo de la caza. Quieren decir que tenía el sobrenombre de Quijada, o Quesada, que en esto hay alguna diferencia en los autores que deste caso escriben; aunque por conjeturas verosímiles se deja entender que se llamaba Quijana. Pero esto importa poco a nuestro cuento: basta que en la narración dél no se salga un punto de la verdad.
    """
    
    # Preprocesar texto
    texto_ejemplo = texto_ejemplo.lower().strip()
    
    # Crear secuencias de caracteres
    longitud_secuencia = 50
    paso = 3
    
    secuencias_texto = []
    siguiente_char = []
    
    for i in range(0, len(texto_ejemplo) - longitud_secuencia, paso):
        secuencias_texto.append(texto_ejemplo[i:i + longitud_secuencia])
        siguiente_char.append(texto_ejemplo[i + longitud_secuencia])
    
    print(f"Número de secuencias: {len(secuencias_texto)}")
    
    # Crear mapeo de caracteres a índices
    chars = sorted(list(set(texto_ejemplo)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    
    print(f"Número de caracteres únicos: {len(chars)}")
    
    # Vectorizar secuencias
    X = np.zeros((len(secuencias_texto), longitud_secuencia, len(chars)), dtype=np.bool_)
    y = np.zeros((len(secuencias_texto), len(chars)), dtype=np.bool_)
    
    for i, secuencia in enumerate(secuencias_texto):
        for t, char in enumerate(secuencia):
            X[i, t, char_indices[char]] = 1
        y[i, char_indices[siguiente_char[i]]] = 1
    
    print("\n=== CONSTRUYENDO MODELO GENERADOR DE TEXTO ===")
    
    # Construir modelo LSTM simple
    model_texto = Sequential([
        LSTM(128, input_shape=(longitud_secuencia, len(chars))),
        Dense(len(chars), activation='softmax')
    ])
    
    # Compilar modelo
    model_texto.compile(
        loss='categorical_crossentropy',
        optimizer='adam'
    )
    
    # Entrenar modelo (pocas épocas para ser rápido)
    model_texto.fit(X, y, batch_size=128, epochs=5)
    
    # Guardar modelo
    model_texto.save('modelo_generador_texto.h5')
    print("Modelo guardado como 'modelo_generador_texto.h5'")
    
    # Función para generar texto
    def generar_texto(modelo, semilla, longitud_generada=200):
        texto_generado = semilla
        
        for i in range(longitud_generada):
            x_pred = np.zeros((1, longitud_secuencia, len(chars)))
            for t, char in enumerate(texto_generado[-longitud_secuencia:]):
                if char in char_indices:
                    x_pred[0, t, char_indices[char]] = 1
            
            probas = modelo.predict(x_pred, verbose=0)[0]
            siguiente_indice = np.random.choice(len(chars), p=probas)
            siguiente_caracter = indices_char[siguiente_indice]
            
            texto_generado += siguiente_caracter
        
        return texto_generado
    
    # Generar ejemplos de texto
    semillas = [
        "en un lugar de la mancha",
        "caballero de noble figura",
        "entre molinos y gigantes"
    ]
    
    print("\n=== EJEMPLOS DE TEXTO GENERADO ===")
    for semilla in semillas:
        texto_generado = generar_texto(model_texto, semilla)
        print(f"\nSemilla: '{semilla}'")
        print(f"Texto generado:\n{texto_generado}")
        
        # Guardar texto generado en archivo
        with open(f"texto_generado_{semilla[:10].replace(' ', '_')}.txt", 'w', encoding='utf-8') as f:
            f.write(texto_generado)
    
    print("\n=== GENERACIÓN DE IMÁGENES SIMPLES ===")
    
    # Crear imágenes abstractas simples usando patrones matemáticos
    def generar_imagen_fractal(tamaño=400, complejidad=0.7, colorido=0.5):
        # Crear malla de coordenadas
        x = np.linspace(-2, 2, tamaño)
        y = np.linspace(-2, 2, tamaño)
        X, Y = np.meshgrid(x, y)
        Z = X + 1j * Y
        
        # Inicializar imagen
        imagen = np.zeros((tamaño, tamaño, 3), dtype=np.uint8)
        
        # Generar patrón fractal simple
        c = complex(-0.8, 0.156)
        
        for i in range(tamaño):
            for j in range(tamaño):
                z = Z[i][j]
                iteraciones = 0
                max_iter = 20 + int(complejidad * 100)
                
                while abs(z) < 10 and iteraciones < max_iter:
                    z = z**2 + c
                    iteraciones += 1
                if iteraciones < max_iter:
                    ratio = iteraciones / max_iter
                    # Asignar colores según iteraciones
                    imagen[i, j, 0] = int(255 * abs(np.sin(ratio * 3.14 * (1 + colorido))))  # R
                    imagen[i, j, 1] = int(255 * abs(np.sin(ratio * 6.28 * colorido)))  # G
                    imagen[i, j, 2] = int(255 * abs(np.cos(ratio * 3.14 * (1 - colorido))))  # B
       
        return imagen
    
    # Generar varias imágenes con diferentes parámetros
    for i, (complejidad, colorido) in enumerate([(0.5, 0.3), (0.7, 0.5), (0.9, 0.7)]):
        nombre_imagen = f"imagen_generada_{i+1}.png"
        
        # Generar imagen
        imagen = generar_imagen_fractal(tamaño=400, complejidad=complejidad, colorido=colorido)
        
        # Guardar imagen
        plt.figure(figsize=(6, 6))
        plt.imshow(imagen)
        plt.axis('off')
        plt.title(f"Imagen generada (c:{complejidad}, col:{colorido})")
        plt.tight_layout()
        plt.savefig(nombre_imagen)
        plt.close()
        print(f"- Imagen guardada como '{nombre_imagen}'")
    
    print("\n=== GENERACIÓN DE CONTENIDO FINALIZADA CON ÉXITO ===")
    print("Archivos generados:")
    print("- modelo_generador_texto.h5 (Modelo generador de texto)")
    print("- texto_generado_*.txt (Textos generados)")
    print("- imagen_generada_*.png (Imágenes generadas)")


## Código principal

In [13]:
# Código principal que permite elegir el módulo a ejecutar
if __name__ == "__main__":
    print("=== SISTEMA DE ML MULTI-MÓDULO ===")
    print("Seleccione el módulo a ejecutar:")
    print("1. Predicción de Series Temporales (Ventas)")
    print("2. Clasificación de Imágenes (MNIST)")
    print("3. Análisis de Sentimientos")
    print("4. Generación de Contenido")
    
    try:
        opcion = int(input("\nIngrese el número del módulo (1-4): "))
        ejecutar_modulo(opcion)
    except ValueError:
        print("Por favor ingrese un número válido (1-4)")
    except KeyboardInterrupt:
        print("\nEjecución interrumpida por el usuario")
    finally:
        print("\n=== PROGRAMA FINALIZADO ===")


=== SISTEMA DE ML MULTI-MÓDULO ===
Seleccione el módulo a ejecutar:
1. Predicción de Series Temporales (Ventas)
2. Clasificación de Imágenes (MNIST)
3. Análisis de Sentimientos
4. Generación de Contenido

=== EJECUTANDO MÓDULO 4: GENERACIÓN DE CONTENIDO ===
=== IMPLEMENTANDO GENERADOR DE TEXTO SIMPLE ===
Número de secuencias: 385
Número de caracteres únicos: 35

=== CONSTRUYENDO MODELO GENERADOR DE TEXTO ===
Epoch 1/5


c:\Users\Javier\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 3.5448
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 3.4957
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 3.4206
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 3.1121
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 3.0176


Modelo guardado como 'modelo_generador_texto.h5'

=== EJEMPLOS DE TEXTO GENERADO ===

Semilla: 'en un lugar de la mancha'
Texto generado:
en un lugar de la mancha ood haooe;ñlo l nxe  e  olas vasooéaxeecn onoolaa  dayodo lomgórooo e lnaxsonr o oaru lélqoea oenmoréhooov  pjos slnot looo adoa v úooa ooo oooooo l gh d ov vna moo  fa;aotgsbl;oéodo snloo oaoooon oa

Semilla: 'caballero de noble figura'
Texto generado:
caballero de noble figurana o ob n ao o aoa eaoaosjsex omeoo  ooyovoa oo dvmil ee ohoo.olr.ooa ooaoeo sad  nao ec  hlgno lxnoaooná  oo a aonde aozo soolea aalol vuoónx e;  ooo ooeoacsalso   ol vnalo aoloj  l  oéooax vóoéoaio 

Semilla: 'entre molinos y gigantes'
Texto generado:
entre molinos y gigantesoé osuxnéo loaoo oon  dtao;boo i lo aetó oeeotvodoaodoa t n;   ravoionoadsusioln o cp   umhoocooo rnah  o hn zaaa; oy.oooiaaoo esoéoeoo no oov aoaoo nnetoooltoopao ddépoooume ooe o ozjaoooy noq ooe r 

=== GENERACIÓN DE IMÁGENES SIMPLES ===
- Imagen guardada como 'imagen_generada